# ETL project by Erin Buday and Chang Woon Jang

The project is related to patients who have taken Prednisone (Steroid) from 2010 to 2020. We used three datasets: 1) patients.csv (111865 rows), 2) encounters.csv (283853 rows), 3) medications.csv (112271 rows)

The final transformed table has 172 rows.

We calcualted the encounter duration of patients for Prednisone and obtaine the statistical results using describe method. Below is the summary,

======================================

count                          172
mean     8 days 14:42:52.325581395
std      1 days 23:40:20.325822717
min                3 days 08:51:00
25%                7 days 04:50:30
50%                8 days 08:20:00
75%                9 days 20:35:00
max               13 days 17:15:00
Name: ENCOUNTER_DURATION, dtype: object

=======================================

In [1]:
# All dependencies: pandas, numpy, sqlalchemy

import pandas as pd
import numpy as np
from sqlalchemy import create_engine, inspect

In [2]:
# read patients.cvs in which patient information is listed including primary key (Id: Patient ID) 

patients_file = "./patients.csv"
patients_df = pd.read_csv(patients_file, parse_dates=['BIRTHDATE', 'DEATHDATE'])
patients_df.head()

,Id,BIRTHDATE,DEATHDATE,FIRST,LAST,MARITAL,RACE,ETHNICITY,GENDER
0,3d8e57b2-3de5-4fb6-95b4-8370270c5cc2,1943-03-11,NaT,Allyn942,Kreiger457,M,asian,asian_indian,F
1,7f4ea9fb-f436-411e-ab34-e94750edfa93,1980-09-28,NaT,Kieth891,King743,M,white,irish,M
2,553b00b2-347c-48ec-90ee-f59924ff50eb,1973-02-22,NaT,Domenica436,Fadel536,M,white,scottish,F
3,50f799aa-740c-4da9-ab85-49d5cbf4046b,1955-03-30,NaT,Kelly223,Turner526,M,white,irish,M
4,bcef3b7a-0380-4b7e-b2b9-2caf07e41f0e,2009-05-24,NaT,Zack583,Purdy2,NaN,white,german,M


In [3]:
# Create a filtered dataframe from specific columns
patients_cols = ["Id", "BIRTHDATE", "RACE", "ETHNICITY", "GENDER"]
patients_transformed = patients_df[patients_cols].copy()

# Rename the column headers
patients_transformed = patients_transformed.rename(columns={"Id": "PATIENTS_ID"})

# Clean the data by dropping duplicates and setting the index
patients_transformed.drop_duplicates("PATIENTS_ID", inplace=True)

patients_transformed

,PATIENTS_ID,BIRTHDATE,RACE,ETHNICITY,GENDER
0,3d8e57b2-3de5-4fb6-95b4-8370270c5cc2,1943-03-11,asian,asian_indian,F
1,7f4ea9fb-f436-411e-ab34-e94750edfa93,1980-09-28,white,irish,M
2,553b00b2-347c-48ec-90ee-f59924ff50eb,1973-02-22,white,scottish,F
3,50f799aa-740c-4da9-ab85-49d5cbf4046b,1955-03-30,white,irish,M
4,bcef3b7a-0380-4b7e-b2b9-2caf07e41f0e,2009-05-24,white,german,M
...,...,...,...,...,...
11859,3e538a8e-2560-40d0-a203-c6cbee9c7ad5,1913-12-24,white,american,M
11860,c9889bfa-3da7-4056-b3d3-9c05737bb378,1913-12-24,white,american,M
11861,0cfef5b2-f5c1-47d7-add0-b29bb167aab5,1913-12-24,white,american,M
11862,e779a47b-572d-4d8d-b152-c8b424d8cb29,1913-12-24,white,american,M


In [4]:
encounters_file = "./encounters.csv"
encounters_df = pd.read_csv(encounters_file, parse_dates=['START', 'STOP'])
encounters_df.head()

,Id,START,STOP,PATIENT,REASONCODE,REASONDESCRIPTION
0,2de9fa09-c147-4716-ae1e-db8df726be21,2019-07-08 13:36:00,2019-07-15 15:54:00,4f400e3f-9be3-41d6-8c29-ac0fe1f44446,72892002.0,Normal pregnancy
1,c97753e6-f8b7-4b58-9768-5351e98d6800,2019-07-08 13:19:00,2019-07-21 15:54:00,49358964-a7de-4888-9b1d-6f0b56f34d35,NaN,NaN
2,301037c2-7d61-411f-878d-f0165121bb1b,2019-07-08 12:46:00,2019-07-20 12:31:00,6194c3a5-0455-40ee-bbea-f8fb02231cde,NaN,NaN
3,95ba474b-941e-457b-8441-4c0205a2c65c,2019-07-08 12:41:00,2019-07-18 07:23:00,f77f265d-6388-481c-82e4-cdd2eb10f51b,NaN,NaN
4,9b50bcf2-36f1-4dc6-adf5-e1654fcd679a,2019-07-08 12:07:00,2019-07-16 00:09:00,c46d742a-6d48-4ebe-b2da-10e5889aa139,NaN,NaN


In [5]:
# Create a filtered dataframe from specific columns
encounters_cols = ["Id", "START", "STOP", "PATIENT"]
encounters_transformed = encounters_df[encounters_cols].copy()

# Rename the column headers
encounters_transformed = encounters_transformed.rename(columns={"Id": "ENCOUNTERS_ID",
                                                               "START": "ENCOUNTER_START",
                                                               "STOP": "ENCOUNTER_STOP",
                                                               "PATIENT": "PATIENTS_ID"})

encounters_transformed

,ENCOUNTERS_ID,ENCOUNTER_START,ENCOUNTER_STOP,PATIENTS_ID
0,2de9fa09-c147-4716-ae1e-db8df726be21,2019-07-08 13:36:00,2019-07-15 15:54:00,4f400e3f-9be3-41d6-8c29-ac0fe1f44446
1,c97753e6-f8b7-4b58-9768-5351e98d6800,2019-07-08 13:19:00,2019-07-21 15:54:00,49358964-a7de-4888-9b1d-6f0b56f34d35
2,301037c2-7d61-411f-878d-f0165121bb1b,2019-07-08 12:46:00,2019-07-20 12:31:00,6194c3a5-0455-40ee-bbea-f8fb02231cde
3,95ba474b-941e-457b-8441-4c0205a2c65c,2019-07-08 12:41:00,2019-07-18 07:23:00,f77f265d-6388-481c-82e4-cdd2eb10f51b
4,9b50bcf2-36f1-4dc6-adf5-e1654fcd679a,2019-07-08 12:07:00,2019-07-16 00:09:00,c46d742a-6d48-4ebe-b2da-10e5889aa139
...,...,...,...,...
283847,edaf52e3-575b-4998-b1e9-bd42743d5aa5,2010-01-01 01:41:00,2010-01-14 01:26:00,fc59091d-7828-487f-92f5-845cc0500db7
283848,7675b948-4b75-482c-a8d3-96a53847616c,2010-01-01 01:29:00,2010-01-10 16:31:00,67113b51-56e7-441c-a45e-5a1f03f445a2
283849,38461ac0-b72d-4fbc-8da2-24082922d885,2010-01-01 00:13:00,2010-01-09 17:18:00,23c7876e-fe60-44a6-b620-2cdb7c1c9ee2
283850,62a03310-9497-4645-9c48-9635acb1cdef,2010-01-01 00:13:00,2010-01-07 08:32:00,23c7876e-fe60-44a6-b620-2cdb7c1c9ee2


In [6]:
# read medications.csv
medications_file = "./medications.csv"
medications_df = pd.read_csv(medications_file)
medications_df.head()

,START,STOP,PATIENT,ENCOUNTER,CODE,DESCRIPTION,REASONCODE,REASONDESCRIPTION
0,9/20/1997,NaN,2043c57c-1085-45a3-87e8-a223d5b6693d,3710b40a-bcc0-4c0f-a204-e47a5f3aec98,316049,Hydromorphone 325 MG,59621000.0,Pain Relief
1,4/19/1957,NaN,50f799aa-740c-4da9-ab85-49d5cbf4046b,64b4efb0-26ba-4c7f-91c1-64d8f62e49e3,1049630,diphenhydrAMINE Hydrochloride 25 MG Oral Tablet,NaN,NaN
2,8/29/2008,8/24/2009,553b00b2-347c-48ec-90ee-f59924ff50eb,ba6137e3-c8cd-42e6-9a02-eafa13d1cfa9,749785,Ortho Tri-Cyclen 28 Day Pack,NaN,NaN
3,10/9/1999,NaN,7f4ea9fb-f436-411e-ab34-e94750edfa93,8e2e1683-82da-48af-a740-80e9ce8e38ca,999969,Amlodipine 5 MG / Fentanyl 100 MCG / Olmesarta...,59621000.0,Pain Relief
4,7/31/2003,NaN,3d8e57b2-3de5-4fb6-95b4-8370270c5cc2,82e9e0a1-3ef3-4cb8-b971-3e3de33652a1,904419,Alendronic acid 10 MG Oral Tablet,NaN,NaN


In [7]:
# Create a filtered dataframe from specific columns
medications_cols = ["PATIENT", "ENCOUNTER", "DESCRIPTION"]
medications_transformed = medications_df[medications_cols].copy()

# Rename the column headers
medications_transformed = medications_transformed.rename(columns={"START": "MEDICATION_START",
                                                                 "STOP": "MEDICATION_STOP",
                                                                 "PATIENT": "PATIENTS_ID",
                                                                 "ENCOUNTER": "ENCOUNTERS_ID"})

medications_transformed

,PATIENTS_ID,ENCOUNTERS_ID,DESCRIPTION
0,2043c57c-1085-45a3-87e8-a223d5b6693d,3710b40a-bcc0-4c0f-a204-e47a5f3aec98,Hydromorphone 325 MG
1,50f799aa-740c-4da9-ab85-49d5cbf4046b,64b4efb0-26ba-4c7f-91c1-64d8f62e49e3,diphenhydrAMINE Hydrochloride 25 MG Oral Tablet
2,553b00b2-347c-48ec-90ee-f59924ff50eb,ba6137e3-c8cd-42e6-9a02-eafa13d1cfa9,Ortho Tri-Cyclen 28 Day Pack
3,7f4ea9fb-f436-411e-ab34-e94750edfa93,8e2e1683-82da-48af-a740-80e9ce8e38ca,Amlodipine 5 MG / Fentanyl 100 MCG / Olmesarta...
4,3d8e57b2-3de5-4fb6-95b4-8370270c5cc2,82e9e0a1-3ef3-4cb8-b971-3e3de33652a1,Alendronic acid 10 MG Oral Tablet
...,...,...,...
112265,2e7ec1d2-15ef-4de0-abb7-44dd9f7f79f6,be72c15e-f1ce-4278-b6f6-c17c17dba3ab,Simvistatin 10 MG
112266,2e7ec1d2-15ef-4de0-abb7-44dd9f7f79f6,52f853cf-7fd7-417d-a81d-e3e5f0599c37,Simvistatin 10 MG
112267,2e7ec1d2-15ef-4de0-abb7-44dd9f7f79f6,e255b9d2-8b0c-4ee8-a622-bfdbc81026e2,Amoxicillin 250 MG / Clavulanate 125 MG Oral T...
112268,2e7ec1d2-15ef-4de0-abb7-44dd9f7f79f6,2e2f3fd5-a2b6-43d9-8799-52c58fc389e0,Simvistatin 10 MG


In [8]:
# first merge two data sets: 1) encounter_transformed, 2) patients_transformed
patient_encounter_df = encounters_transformed.merge(patients_transformed, on=['PATIENTS_ID'])
patient_encounter_df.head()

,ENCOUNTERS_ID,ENCOUNTER_START,ENCOUNTER_STOP,PATIENTS_ID,BIRTHDATE,RACE,ETHNICITY,GENDER
0,2de9fa09-c147-4716-ae1e-db8df726be21,2019-07-08 13:36:00,2019-07-15 15:54:00,4f400e3f-9be3-41d6-8c29-ac0fe1f44446,1999-06-07,white,english,F
1,854ef51e-dce1-45d0-be4f-4d9513139c4d,2019-06-10 13:36:00,2019-06-19 13:20:00,4f400e3f-9be3-41d6-8c29-ac0fe1f44446,1999-06-07,white,english,F
2,247f124f-cd94-464c-ab1d-4afe0e3b7a95,2019-05-13 13:36:00,2019-05-24 15:43:00,4f400e3f-9be3-41d6-8c29-ac0fe1f44446,1999-06-07,white,english,F
3,3a7355c0-0311-4481-b3bd-01a9a0c422c8,2019-04-18 13:36:00,2019-04-28 10:20:00,4f400e3f-9be3-41d6-8c29-ac0fe1f44446,1999-06-07,white,english,F
4,fab9d163-0e22-424e-b548-106e229a5111,2019-04-15 13:36:00,2019-04-26 03:50:00,4f400e3f-9be3-41d6-8c29-ac0fe1f44446,1999-06-07,white,english,F


In [9]:
# extract data for predniSONE 2.5 MG [Deltasone], predniSONE 5 MG Oral Tablet

med_predniSONE_df = medications_transformed.loc[(medications_transformed.DESCRIPTION == "predniSONE 2.5 MG [Deltasone]") | 
                                       (medications_transformed.DESCRIPTION == "predniSONE 5 MG Oral Tablet")]

med_predniSONE_df.sort_values(by='DESCRIPTION')

,PATIENTS_ID,ENCOUNTERS_ID,DESCRIPTION
94284,0c26b29a-d073-47fb-9d29-1f7e108ca6e3,5ab4ee4d-c22a-452f-aabd-169b6b109ea7,predniSONE 2.5 MG [Deltasone]
89386,a95979cd-50d3-44bf-9693-61834bf5fb7e,24b10d4d-c5d7-439d-89a6-20d77f583252,predniSONE 2.5 MG [Deltasone]
19507,01adbd20-7e2a-40c3-b5fd-e8329226f0d0,817424c9-46b4-4786-93d6-f892bad927ec,predniSONE 2.5 MG [Deltasone]
19512,01adbd20-7e2a-40c3-b5fd-e8329226f0d0,ed05ca22-2c6c-4490-98a9-99b9e0152dc6,predniSONE 2.5 MG [Deltasone]
19516,01adbd20-7e2a-40c3-b5fd-e8329226f0d0,f6c924eb-b60b-4ca2-9a03-7d04027a0a00,predniSONE 2.5 MG [Deltasone]
...,...,...,...
38385,6bd86923-06a1-43ff-9958-d24d50f6e275,6b9db7b0-1e27-401c-b954-9d83c164f831,predniSONE 5 MG Oral Tablet
41785,97d3e3f3-fc63-4aed-b7f0-d9d3e2563ef5,9c53bc54-309a-4714-a3f6-cb7bed950511,predniSONE 5 MG Oral Tablet
43412,d498303c-6a6d-4fe5-b95a-eea3caf65120,5ecdc916-30ac-4741-bb84-68ec2596b6e3,predniSONE 5 MG Oral Tablet
35820,2ae76158-a092-4c7e-8b96-685b689fae29,ea726692-acf0-44c6-b1b3-ab4355fc5c61,predniSONE 5 MG Oral Tablet


In [10]:
# merge medications_transformed with patient_encounter_df and make final data frame called patientsprednisone

patientsprednisone = med_predniSONE_df.merge(patient_encounter_df, on=['PATIENTS_ID', 'ENCOUNTERS_ID'])

patientsprednisone.sort_values(by='ENCOUNTER_START', ascending=True).head()

,PATIENTS_ID,ENCOUNTERS_ID,DESCRIPTION,ENCOUNTER_START,ENCOUNTER_STOP,BIRTHDATE,RACE,ETHNICITY,GENDER
128,e88001de-d8f7-4bd9-b1c3-b521572e5f9c,ec75c18d-618b-495d-85c3-659efe8bf6c5,predniSONE 5 MG Oral Tablet,2010-03-12 13:22:00,2010-03-21 04:23:00,1981-03-08,white,greek,F
33,9e6ba167-ce60-4a50-a9e6-8e281a21cbd5,a87ca735-1720-4818-bcdd-de97127197f9,predniSONE 5 MG Oral Tablet,2010-03-27 10:36:00,2010-04-01 03:46:00,1993-11-21,white,irish,M
130,44887bbd-e934-41c4-badb-383a02df47ea,0369886e-b62b-4784-9f16-59799caf33f4,predniSONE 5 MG Oral Tablet,2010-05-13 23:40:00,2010-05-21 10:21:00,2009-03-14,white,english,M
76,322a3970-7322-40bd-878e-151f1f9a352e,9d3f19be-3d37-49d8-bf2d-c10a7a875301,predniSONE 5 MG Oral Tablet,2010-05-21 15:47:00,2010-05-30 11:37:00,2009-05-26,black,dominican,M
10,81b2e402-2884-44b4-a77e-4918aaf00852,150d8510-6816-4673-9e44-e004ac87e2f0,predniSONE 5 MG Oral Tablet,2010-06-21 13:42:00,2010-06-28 17:49:00,2009-01-27,hispanic,puerto_rican,M


In [11]:
# calculate the encounter duration
patientsprednisone['ENCOUNTER_DURATION'] = patientsprednisone['ENCOUNTER_STOP']\
                                        - patientsprednisone['ENCOUNTER_START']

# statistics: encounter duration for patients who took prednisone of 2.5 MG and 5.0 MG
print(patientsprednisone['ENCOUNTER_DURATION'].describe())

#change timedelta to str in order to load datafram into database via sqlachemy
patientsprednisone['ENCOUNTER_DURATION'] = patientsprednisone['ENCOUNTER_DURATION'].astype(str).str[-18:-10]

patientsprednisone

count                          172
mean     8 days 14:42:52.325581395
std      1 days 23:40:20.325822717
min                3 days 08:51:00
25%                7 days 04:50:30
50%                8 days 08:20:00
75%                9 days 20:35:00
max               13 days 17:15:00
Name: ENCOUNTER_DURATION, dtype: object


,PATIENTS_ID,ENCOUNTERS_ID,DESCRIPTION,ENCOUNTER_START,ENCOUNTER_STOP,BIRTHDATE,RACE,ETHNICITY,GENDER,ENCOUNTER_DURATION
0,bddcb901-8eab-4491-8027-04e4a81f405e,794b3c7a-c586-434b-ac48-b141b62b586b,predniSONE 5 MG Oral Tablet,2013-04-15 14:00:00,2013-04-24 23:36:00,2012-10-17,white,irish,M,9 day
1,f545edc5-52be-4c4d-b070-9c71bfc4af23,f88838e7-92b7-4a20-a3d9-d850db048604,predniSONE 5 MG Oral Tablet,2018-03-30 01:35:00,2018-04-05 13:03:00,2012-06-28,white,irish,F,6 day
2,e75bb0f6-201e-4a56-a00f-0b1b5555926b,7384f92f-ac42-4bfc-9755-8d022a914314,predniSONE 5 MG Oral Tablet,2017-12-22 01:38:00,2017-12-31 15:53:00,2016-11-26,white,german,F,9 day
3,6e9bb51d-e1c9-4d10-8462-854de9a8a4e2,8999a87f-f6fa-4714-badb-b42a65c9daf7,predniSONE 5 MG Oral Tablet,2013-06-28 13:14:00,2013-07-04 07:21:00,2011-03-11,white,irish,M,5 day
4,8a6e8ea2-9b6e-4241-9d33-51eb29a1429b,fec27ba2-8396-48f8-8581-be3fa0388fa3,predniSONE 5 MG Oral Tablet,2014-05-05 04:55:00,2014-05-11 14:53:00,2013-11-05,white,irish,M,6 day
...,...,...,...,...,...,...,...,...,...,...
167,169d2b1a-60ab-4e19-8f3a-26377f0eb0e5,ea3ebaa5-6fab-48c4-9ba0-fe05ecedda48,predniSONE 5 MG Oral Tablet,2014-01-26 06:47:00,2014-02-03 10:29:00,2005-10-10,white,portuguese,F,8 day
168,4abf8b3c-43e6-40e1-a70b-9ea43fdbd002,75ba0d22-015d-4d28-8f08-0d2dc8bc01c6,predniSONE 5 MG Oral Tablet,2013-03-10 10:42:00,2013-03-19 16:20:00,2012-02-14,white,american,F,9 day
169,118dd680-f736-4194-bbbb-cf721603899b,27200bce-5852-49e6-b151-425fcba445fd,predniSONE 5 MG Oral Tablet,2012-12-04 21:31:00,2012-12-12 09:59:00,2012-03-09,black,west_indian,M,7 day
170,5bd08389-77d2-4250-aa19-88ef5ed912de,0dd28d1c-aa21-4e41-8070-4417ca1589c6,predniSONE 5 MG Oral Tablet,2016-11-21 06:43:00,2016-11-30 20:01:00,2016-03-26,white,portuguese,F,9 day


In [12]:
#connect local database

rds_connection_string = "postgres:{password}@localhost:5432/customer_db"
engine = create_engine(f'postgresql://{rds_connection_string}')

In [13]:
patientsprednisone.to_sql(name='PrednisonePatients', con=engine, if_exists='append', index=False)

172

In [14]:
engine.table_names()

C:\Users\hyeji\AppData\Local\Temp\ipykernel_31028\4162273999.py:1: SADeprecationWarning: The Engine.table_names() method is deprecated and will be removed in a future release.  Please refer to Inspector.get_table_names(). (deprecated since: 1.4)
  engine.table_names()


['customer_name', 'customer_location', 'PrednisonePatients']

In [16]:
#sql_com = 'SELECT * FROM "PrednisonePatients"'
pd.read_sql_query('SELECT * FROM "PrednisonePatients" LIMIT 5', con=engine)


,PATIENTS_ID,ENCOUNTERS_ID,DESCRIPTION,ENCOUNTER_START,ENCOUNTER_STOP,BIRTHDATE,RACE,ETHNICITY,GENDER,ENCOUNTER_DURATION
0,bddcb901-8eab-4491-8027-04e4a81f405e,794b3c7a-c586-434b-ac48-b141b62b586b,predniSONE 5 MG Oral Tablet,2013-04-15 14:00:00,2013-04-24 23:36:00,2012-10-17,white,irish,M,9 day
1,f545edc5-52be-4c4d-b070-9c71bfc4af23,f88838e7-92b7-4a20-a3d9-d850db048604,predniSONE 5 MG Oral Tablet,2018-03-30 01:35:00,2018-04-05 13:03:00,2012-06-28,white,irish,F,6 day
2,e75bb0f6-201e-4a56-a00f-0b1b5555926b,7384f92f-ac42-4bfc-9755-8d022a914314,predniSONE 5 MG Oral Tablet,2017-12-22 01:38:00,2017-12-31 15:53:00,2016-11-26,white,german,F,9 day
3,6e9bb51d-e1c9-4d10-8462-854de9a8a4e2,8999a87f-f6fa-4714-badb-b42a65c9daf7,predniSONE 5 MG Oral Tablet,2013-06-28 13:14:00,2013-07-04 07:21:00,2011-03-11,white,irish,M,5 day
4,8a6e8ea2-9b6e-4241-9d33-51eb29a1429b,fec27ba2-8396-48f8-8581-be3fa0388fa3,predniSONE 5 MG Oral Tablet,2014-05-05 04:55:00,2014-05-11 14:53:00,2013-11-05,white,irish,M,6 day
